# 🧠 Optimización del Modelo de Predicción de Salario en IT con Nivel Educativo Ordinal

Este notebook incluye:
- Conversión de `Nivel_educativo` a escala ordinal (0-7)
- Preprocesamiento completo
- Ingeniería de características
- GridSearchCV para LightGBM
- Evaluación del rendimiento final


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")


In [2]:
# Cargar dataset
df = pd.read_csv("./data/survey_results_public.csv", low_memory=False)

# Renombrar columnas clave
columnas = {
    "Employment": "Tipo_empleo",
    "RemoteWork": "Trabajo_remoto",
    "DevType": "Rol",
    "EdLevel": "Nivel_educativo",
    "YearsCodePro": "Anios_experiencia",
    "Country": "Pais",
    "OrgSize": "Tamano_empresa",
    "ConvertedCompYearly": "Salario_anual"
}
df = df[list(columnas.keys())].rename(columns=columnas)

# Filtrar salarios extremos
df = df[df["Salario_anual"].between(10000, 300000)]

# Convertir experiencia a numérico
df["Anios_experiencia"] = df["Anios_experiencia"].replace({
    "Less than 1 year": "0", "More than 50 years": "51"
})
df["Anios_experiencia"] = pd.to_numeric(df["Anios_experiencia"], errors="coerce")

# Agrupar países poco frecuentes
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro")

# Expandir columna multietiqueta 'Rol'
roles = df["Rol"].str.get_dummies(sep=";")
df = pd.concat([df.drop(columns=["Rol"]), roles], axis=1)
df = df.dropna()


In [3]:
# Codificación ordinal 0-7
mapa_educacion = {
    "Primary/elementary school": 0,
    "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)": 1,
    "Some college/university study without earning a degree": 2,
    "Associate degree (A.A., A.S., etc.)": 3,
    "Bachelor’s degree (B.A., B.S., B.Eng., etc.)": 4,
    "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)": 5,
    "Professional degree (JD, MD, etc.)": 6,
    "Other doctoral degree (Ph.D., Ed.D., etc.)": 7
}

df = df[df["Nivel_educativo"].isin(mapa_educacion.keys())]
df["Nivel_educativo_ordinal"] = df["Nivel_educativo"].map(mapa_educacion)


In [4]:
# Transformar salario y generar nuevas variables
df["Salario_anual_log"] = np.log1p(df["Salario_anual"])
df["Anios_experiencia_log"] = np.log1p(df["Anios_experiencia"])
df["Anios_experiencia_cuadrado"] = df["Anios_experiencia"] ** 2


In [5]:
X = df.drop(columns=["Salario_anual", "Salario_anual_log", "Nivel_educativo"])
y = df["Salario_anual_log"]

cat_cols = ["Tipo_empleo", "Trabajo_remoto", "Pais", "Tamano_empresa"]
num_cols = [col for col in X.columns if col not in cat_cols]


In [6]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_cols),
    ("num", num_pipeline, num_cols)
])


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LGBMRegressor(random_state=42))
])

param_grid = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [10, 15],
    "regressor__learning_rate": [0.05, 0.1],
    "regressor__num_leaves": [31, 50],
    "regressor__min_child_samples": [20, 30]
}

search = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring="r2",
    cv=3,
    verbose=2,
    n_jobs=-1
)

search.fit(X_train, y_train)
print("Mejores parámetros encontrados:", search.best_params_)
print("Mejor R² en validación cruzada:", search.best_score_)


Fitting 3 folds for each of 32 candidates, totalling 96 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 334
[LightGBM] [Info] Number of data points in the train set: 15452, number of used features: 77
[LightGBM] [Info] Start training from score 11.105598
Mejores parámetros encontrados: {'regressor__learning_rate': 0.05, 'regressor__max_depth': 15, 'regressor__min_child_samples': 30, 'regressor__n_estimators': 200, 'regressor__num_leaves': 31}
Mejor R² en validación cruzada: 0.5970086026178337


In [8]:
mejor_modelo = search.best_estimator_
y_pred = mejor_modelo.predict(X_test)
print("R² en test:", r2_score(y_test, y_pred))


R² en test: 0.5937980451313298
